In [1]:
import sys,os
import pandas as pd
import numpy as np
sys.path.append(os.path.abspath('./../../'))
from cell_extractor.CellDetectorBase import CellDetectorBase
from cell_extractor.AnnotationProximityTool import AnnotationProximityTool
from cell_extractor.diagnostics.ToolKit import get_DataFrame_from_query_result,get_DataFrame_from_detection_df
from cell_extractor.utils import numpy_to_json
from Controllers.MarkedCellController import MarkedCellController
from lib.UrlGenerator import UrlGenerator

In [2]:
base = CellDetectorBase('DK41',round = 3)
controller = MarkedCellController()
sure,unsure,_ = base.get_detection_by_category()
beth_cell = controller.get_marked_cells({'FK_annotator_id':2,'FK_prep_id':'DK41'})
beth_cell = get_DataFrame_from_query_result(beth_cell,'beth_annotation',factor = [0.325,0.325,20])
sure = get_DataFrame_from_detection_df(sure,'round3_sure')
unsure = get_DataFrame_from_detection_df(unsure,'round3_unsure')
print(len(sure),len(unsure),len(beth_cell))

5205 1476 262


In [3]:
tool = AnnotationProximityTool()
tool.set_annotations_to_compare(pd.concat([sure,unsure,beth_cell]))
tool.find_equivalent_points()
tool.print_grouping()

calculating distance matrix
finding points that are close to each other
grouping and labeling points that are close to each other
before removing duplicates 6943
after removing duplicates 6657
(('round3_sure',), 4998)
(('round3_unsure',), 1366)
(('round3_sure', 'beth_annotation_Beth_POSITIVE'), 203)
(('beth_annotation_Beth_POSITIVE',), 29)
(('round3_unsure', 'beth_annotation_Beth_POSITIVE'), 22)
(('round3_unsure', 'round3_unsure'), 20)
(('round3_unsure', 'round3_unsure', 'round3_unsure'), 12)
(('round3_sure', 'beth_annotation_Beth_POSITIVE', 'beth_annotation_Beth_POSITIVE'), 4)
(('round3_unsure', 'round3_unsure', 'round3_unsure', 'round3_unsure'), 3)


In [4]:
possibly_connected = tool.find_annotation_in_category([['round3_sure', 'beth_annotation_Beth_POSITIVE', 'beth_annotation_Beth_POSITIVE']])

In [5]:
sures_json = numpy_to_json(np.array([sure.x,sure.y,sure.section]),color_hex='#c61b2b')
unsures_json = numpy_to_json(np.array([unsure.x,unsure.y,unsure.section]),color_hex='#00a1e5')
possibly_connected_json = numpy_to_json(np.array([possibly_connected.x,possibly_connected.y,possibly_connected.section]),color_hex='#e9dde3')

In [6]:
animal = 'DK41'
urlGen = UrlGenerator()
urlGen.add_stack_image(animal,channel=1)
urlGen.add_stack_image(animal,channel=2,color='red')
urlGen.add_stack_image(animal,channel=3,color='green')
urlGen.add_annotation_layer('Sure',annotations = sures_json)
urlGen.add_annotation_layer('Unsure',annotations = unsures_json)
urlGen.add_annotation_layer('possibly_connected',annotations = possibly_connected_json)
urlGen.add_to_database('DK41 round3 detector possible conntected cell',34)

660